In [ ]:
import wheel
import numpy as np
import sklearn as skl
from mat4py import loadmat
import scipy.io
import csv
import struct
import h5py
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colormaps
#from mne_bids import BIDSPath, read_raw_bids
from scipy.signal import butter, filtfilt,welch
from scipy.stats import zscore 
#from mne.time_frequency import tfr_multitaper
import seaborn as sb
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import tkinter as tk
#import mne
#from mne.viz import plot_alignment, snapshot_brain_montage
#import mne_bids
#from mne.channels import make_standard_montage

from sklearn.model_selection import train_test_split
import pickle
import os
import dask
import dask.dataframe as dd


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colormaps
from mne_bids import BIDSPath, read_raw_bids
from scipy.signal import butter, filtfilt
from mne.time_frequency import tfr_multitaper
import seaborn as sb
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import tkinter as tk



import mne
from mne.viz import plot_alignment, snapshot_brain_montage
import mne_bids
from mne.channels import make_standard_montage

In [ ]:
import pandas as pd

In [ ]:
import warnings
warnings.filterwarnings('once')

In [ ]:
"""
This file is to be run after the pooling file. The goal of it is to concatenate all of the dataframes so we have 3 seperate frames
one of GNS, one for GS, and one for NGNS
"""

In [ ]:
#Upload and pool all the subject data
sub_list = ['S2', 'S4', 'S5', 'S6', 'S7', 'S8','S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16']

In [ ]:
concat_ng = []
concat_gns = []
concat_ngns = []
remain_ng = []
remain_gns = []
remain_ngns = []

In [ ]:
GS = 'GS'
GNS = 'GNS'
NGNS = 'NGNS'

In [ ]:
#Import in elec_table so only valid electrodes remain in the analysis<- change paths to meet your directory structure

In [ ]:
elec_path = "/Volumes/Expansion/4WT/COLLAB-CODE/Alliyah/Sub-Mat-Converted/elecFinal.mat" #This will stay static for every participant once its changed to the correct path                          

In [ ]:
elec_raw = scipy.io.loadmat(elec_path, squeeze_me=True, simplify_cells = True, struct_as_record = False)

In [ ]:
elec_frame = pd.DataFrame(elec_raw['elec_out'])

In [ ]:
#For loop to concatenate files, remove rejects, and delete any NA rows
#Print statements available for debugging can be commented out if unneccesary 
for i,sub in enumerate(sub_list):
    #Create standard path
    gen_path = './'+sub+'_' 
    add_path = '_Correct'
    #Create three generated paths
    gs_path = gen_path+GS+add_path+'.csv.gz'
    gns_path = gen_path+GNS+add_path+'.csv.gz'
    ngns_path = gen_path+NGNS+add_path+'.csv.gz'
    #load files
    gs_frame = pd.read_csv(gs_path)
    gns_frame = pd.read_csv(gns_path)
    ngns_frame = pd.read_csv(ngns_path)
    #Add subject id
    sub_id = int(sub[1:])
    print('This is id', sub_id)
    gs_frame['sub_id']= sub_id
    gns_frame['sub_id'] = sub_id
    ngns_frame['sub_id'] = sub_id
    #remove reject electrodes
    reject_data = list(elec_frame[(elec_frame['subjectNum'] == sub_id) & (
                          (elec_frame['isRejectFSL'] == 1) |
                          (elec_frame['isRejectManual'] == 1) | (elec_frame['isRejectFSL'] == 1))]['elecNum'])
    
    reject_data_new = list(set(reject_data) & set(gs_frame.index))
    if(len(reject_data_new) < len(reject_data)):
        print(sub, "Mismatch")
        print(len(reject_data))
        print(len(reject_data_new))
    print(reject_data)
    gs_frame.drop(reject_data_new,inplace = True)
    gns_frame.drop(reject_data_new, inplace = True)
    ngns_frame.drop(reject_data_new, inplace = True)
    #remove Na's
    gs_frame.replace(np.nan, pd.NA).dropna(axis=1)
    gns_frame.replace(np.nan, pd.NA).dropna(axis=1)
    ngns_frame.replace(np.nan, pd.NA).dropna(axis=1)
    
    gs_frame.dropna(how = 'any' ,inplace = True)
    gns_frame.dropna(how = 'any' ,inplace = True)
    ngns_frame.dropna(how = 'any' ,inplace = True)
    #check
    #Add to lists
   
    concat_ng.append(gs_frame)
    concat_gns.append(gns_frame)
    concat_ngns.append(ngns_frame)
  

In [ ]:
#Code underneath makes sure columns align before attempting final concats

In [ ]:
selected_columns = [str(i) for i in range(151)] + ['sub_id']

selected_columns

In [ ]:
# Concatenate the selected columns from each DataFrame
gs_elec = pd.concat([df[selected_columns] for df in concat_ng])
gns_elec = pd.concat([df[selected_columns] for df in concat_gns])

In [ ]:
for df in concat_ngns:
    if '150' not in df.columns:
        print(df['sub_id'][0])

In [ ]:
ngns_elec = pd.concat([df[selected_columns] for df in concat_ngns])

In [ ]:
gs_elec.dropna(how = 'any', inplace = True)
gns_elec.dropna(how = 'any', inplace = True)
ngns_elec.dropna(how = 'any', inplace = True)

In [ ]:
gs_elec.isna().count()

In [ ]:
ngns_elec

In [ ]:
#Remove the rejects

In [ ]:
gs_elec['type'] = 'GS'
gns_elec['type'] = 'GNS'
ngns_elec['type'] = 'NGNS'

In [ ]:
gs_elec.isna().count()

In [ ]:
gs_elec.to_csv('GS_corr.csv.gz', compression = 'gzip')
gns_elec.to_csv('GNS_corr.csv.gz', compression = 'gzip')
ngns_elec.to_csv('NGNS_corr.csv.gz', compression = 'gzip')

In [ ]:
gns_elec

In [ ]:
ngns_elec

In [ ]:
#Below here is deprecated code that could be ulitized to attempt to include subjects 1 and 3 <- is not fully functional though

In [ ]:
lister = remain_gns[1:4]

In [ ]:
lister.append(remain_gns[5])

In [ ]:
analyze = pd.concat(lister, ignore_index = True)

In [ ]:
analyze

In [ ]:
list_154 = [remain_gns[4], remain_gns[6]]

In [ ]:
analyze2 = pd.concat(list_154, ignore_index = True)

In [ ]:
list_154_2 = [remain_gns[4], remain_gns[6]]

In [ ]:
analyze3 = pd.concat(list_154_2, ignore_index = True)

In [ ]:
list_154_3 = [remain_ngns[4], remain_ngns[6]]
analyze4 = pd.concat(list_154_3, ignore_index = True)

In [ ]:
analyze2

In [ ]:
list(analyze2.columns)

In [ ]:
analyze2.drop(['Unnamed: 0'],axis = 1, inplace = True)

In [ ]:
analyze2.shape

In [ ]:
analyze3.drop(['Unnamed: 0'],axis = 1, inplace = True)
analyze4.drop(['Unnamed: 0'],axis = 1, inplace = True)

In [ ]:
analyze2['type'] = 'NG'
analyze3['type'] = 'GNS'
analyze4['type'] = 'NGNS'

In [ ]:
ng_elec_pool = pd.concat([ng_elec, analyze2], ignore_index = True)
gns_elec_pool = pd.concat([gns_elec, analyze3], ignore_index = True)
ngns_elec_pool = pd.concat([ngns_elec, analyze4], ignore_index = True)

In [ ]:
ng_elec_pool

In [ ]:
remain_ng.pop(6)

In [ ]:
len(remain_ng)

In [ ]:
remain_ng.pop(4)

In [ ]:
for frame in remain_ng:
    print('sub ',frame.loc[0,'sub_id'])
    print(frame.shape)

In [ ]:
analyze.columns

In [ ]:
analyze.drop(['Unnamed: 0','151'152', '153', '154', '155', '156'], axis = 1, inplace = True)

In [ ]:
analyze.shape

In [ ]:
analyze['Type'] = 'NG'

In [ ]:
analyze

In [ ]:
gns_elec_pool

In [ ]:
len(remain_gns)

In [ ]:
gns_elec_pool

In [ ]:
ngns_elec_pool['sub_id'].value_counts()

In [ ]:
ng_elec_pool_total

In [ ]:
ngns_elec_pool

In [ ]:
lister = remain_gns[1:4]

In [ ]:
lister.append(remain_gns[5])

In [ ]:
analyze = pd.concat(lister, ignore_index = True)

In [ ]:
remain_gns.pop(6)

In [ ]:
remain_gns.pop(4)

In [ ]:
analyze.drop(['Unnamed: 0','151','152', '153', '154', '155', '156'], axis = 1, inplace = True)

In [ ]:
analyze['type'] = 'GNS'

In [ ]:
gns_elec_pool_total = pd.concat([gns_elec_pool, analyze], ignore_index = True)

In [ ]:
gns_elec_pool_total

In [ ]:
gns_elec_pool_total.to_csv('gns_elec_total.csv.gz', compression = 'gzip')

In [ ]:
#Behavioral

In [ ]:
sentence_frame = pd.read_csv('sentence_data.csv.gz', compression = 'gzip')

In [ ]:
sentence_frame.drop(['Unnamed: 0'], axis = 1, inplace = True)